In [1]:
import pandas as pd

In [2]:
b4_sign_df = pd.read_csv('data/b4_sign_v1.csv')

In [4]:
b4_sign_df.columns

Index([u'distinct_id', u'event', u'browser', u'browser_version',
       u'first_browser_language', u'first_referrer_host', u'first_visit_time',
       u'ip', u'is_first_day', u'is_first_time', u'latest_utm_campaign',
       u'latest_utm_content', u'latest_utm_medium', u'latest_utm_source',
       u'latest_utm_term', u'model', u'os', u'os_version', u'referrer_host',
       u'screen_height', u'screen_width', u'title', u'current_page',
       u'utm_campaign', u'utm_content', u'utm_medium', u'utm_source',
       u'utm_term', u'session_referrer_host', u'company', u'contact', u'email',
       u'url_to_submit', u'isMsg', u'isSuccess', u'name', u'btnClick_page',
       u'pagePosition', u'pageStayTime', u'project_name', u'referrHostUrl',
       u'requestBtn', u'result', u'session_page_url', u'site_url',
       u'url_on_page_close', u'time', u'Bj_timestamp', u'dayofweek',
       u'hourofday', u'date'],
      dtype='object')

# select/create useful features used in flating data

### distinct_id

### event， pageStageTime, current_page

In [5]:
# b4_sign_df.event.value_counts()

In [6]:
# b4_sign_df.current_page.value_counts(dropna = False)

In [7]:
# b4_sign_df[['event','current_page','pageStayTime']]

In [8]:
# b4_sign_df[(b4_sign_df.pageStayTime.notnull())][['event','current_page']]

### for event:
    event_to_sign:              
        click_send_cellphone:    ---> result: # of failure, whether success(0/1)
        verify_cellphone_code:   ---> result: # of failure, whether success(0/1)
        clickSubmit:             ---> may be dropped
        formSubmit:              ---> isSuccess: # of failure, whether success(0/1)
    event_b4_sign:
        $pageview                ---> look into current_page column
        btnClick                 ---> page, name
        index_leave              ---> pageStayTime, pagePosition
        demo_leave               ---> pageStayTime, pagePosition
        about_leave              ---> pageStayTime, pagePosition
        courses_leave            ---> pageStayTime, pagePosition
        courses_play_leave       ---> pageStayTime, pagePosition
    other:
        profile_set_once         ---> if someone has this column, it's a new user, otherwise not (0/1)
        page_close               ---> pageStayTime,?? but when does it happen, url_on_page_close, for user signing up or not?

### for current_page: different degree to which a user to sign up, a feature or column is active_sign
    page with interface to sign up: score 1
        homepage    
        demo        
        about
        play_video
    page without interface to sign up: score 0
        lookmanual  ---> 
        manual
        product
        index
    others: score
        tool         
        bround          
        share             
        translate_c --> delete     
        local_host: ----> homepage

In [9]:
b4_sign_df[(b4_sign_df.current_page == 'translate_c')].distinct_id

22647    c86f9a49f36a1b1a8ad8176b5db44724334748c2
Name: distinct_id, dtype: object

In [10]:
b4_sign_df.drop(labels = 22647, axis = 0, inplace = True)

In [11]:
b4_sign_df.reset_index(drop = True, inplace = True)

In [12]:
b4_sign_df.loc[(b4_sign_df.current_page == 'local_host'),'current_page'] = 'homepage'

In [13]:
b4_sign_df.current_page.value_counts(dropna = False)

None          26250
homepage       8345
lookmanual     1965
demo           1867
manual         1701
product        1595
index          1421
bround         1158
about           770
courses         752
play_video      547
tool            151
share            20
Name: current_page, dtype: int64

### browser

In [14]:
b4_sign_df.browser.value_counts(dropna = False)

chrome            25494
NaN                6481
safari             4340
firefox            2422
qq                 1920
sogou              1604
ie                 1454
edge                727
uc                  632
micromessenger      522
webview             203
maxthon             179
mi                  105
baidu                85
theworld             82
liebao               71
360                  59
na                   59
opera                46
android              33
tt                   15
coolnovo              4
yandex                4
ali-ap                1
Name: browser, dtype: int64

#### classify them into some main group:
    chrom, safari, firefox, qq, sogou, ie, others

In [15]:
b4_sign_df.browser.fillna('None',inplace = True)

In [16]:
b4_sign_df.loc[(b4_sign_df.browser.apply(lambda x: x not in ['chrome','safari','firefox','qq','sogou','ie', 'None'])),'browser'] = 'others'

In [17]:
b4_sign_df.browser.value_counts(dropna = False)

chrome     25494
None        6481
safari      4340
others      2827
firefox     2422
qq          1920
sogou       1604
ie          1454
Name: browser, dtype: int64

### for first_visit_time, just with profile_set_once, not use now

In [18]:
# b4_sign_df.first_visit_time.value_counts(dropna=False)
# b4_sign_df[(b4_sign_df.first_visit_time.notnull())].event.value_counts(dropna = False)

In [19]:
# b4_sign_df[(b4_sign_df.event == 'profile_set_once')].tail(1)

In [20]:
# b4_sign_df[(b4_sign_df.distinct_id == '7b0215cb110a50650cb3900532fb859ebdceeb9d')]

In [21]:
# b4_sign_df.first_visit_time.value_counts(dropna = False)

### for ip, maybe we can convert it to district, district is a new feature

In [22]:
# b4_sign_df[(b4_sign_df.ip.notnull())].event.value_counts(dropna = False)

### for latest_utm_term

In [23]:
# b4_sign_df.latest_utm_term.value_counts(dropna = False)

### for u'latest_utm_campaign'

In [24]:
# b4_sign_df[(b4_sign_df.latest_utm_campaign.notnull())].event.value_counts(dropna = False)

In [25]:
# b4_sign_df.latest_utm_campaign.value_counts(dropna = False)

### for u'latest_utm_content'

In [26]:
# b4_sign_df.latest_utm_content.value_counts(dropna = False)

### for model and os

In [27]:
# pc, mac, phone,
# na???
# samsung --> phone
b4_sign_df.model.value_counts(dropna = False)

pc         24936
mac        11796
NaN         6481
iphone      1437
android      487
huawei       352
na           299
vivo         188
samsung      133
meizu        125
mi           113
ipad          89
nexus         49
hongmi        30
lenovo         9
coolpad        8
htc            4
oppo           3
zte            2
lg             1
Name: model, dtype: int64

In [28]:
# b4_sign_df.model.fillna('None',inplace = True)

In [29]:
# b4_sign_df.loc[(b4_sign_df.model.apply(lambda x: x not in ['pc','mac','iphone','ipad','android','None'])),'model'] = 'android'

In [30]:
b4_sign_df.os.value_counts(dropna = False)

windows    24936
macosx     11796
NaN         6481
iOS         1526
Android     1504
linux        267
na            32
Name: os, dtype: int64

### for title, seems to be overlap with current_page, drop it

In [31]:
# b4_sign_df[(b4_sign_df.title.notnull())].event.value_counts(dropna = False)

In [32]:
# b4_sign_df.title.value_counts(dropna = False)

In [33]:
# b4_sign_df.current_page.value_counts(dropna = False)

### for u'utm_campaign', u'utm_content', u'utm_medium', u'utm_source',u'utm_term'

In [34]:
b4_sign_df.utm_source.value_counts(dropna = False)

None      36846
baidu      8540
others      908
google      184
wechat       64
Name: utm_source, dtype: int64

In [35]:
b4_sign_df[(b4_sign_df.utm_source != 'None')].event.value_counts(dropna = False)

$pageview           5469
profile_set_once    4227
Name: event, dtype: int64

### combine latest_utm_source and utm_source

In [36]:
b4_sign_df.latest_utm_source.value_counts(dropna = False)

None      22213
baidu     21710
others     2089
wechat      275
google      255
Name: latest_utm_source, dtype: int64

In [37]:
b4_sign_df[(b4_sign_df.latest_utm_source != 'None')].event.value_counts(dropna = False)

$pageview                11731
btnClick                  5146
index_leave               4762
demo_leave                 619
about_leave                368
formSubmit                 353
courses_leave              289
click_send_cellphone       274
courses_play_leave         271
verify_cellphone_code      263
clickSubmit                213
page_close                  40
Name: event, dtype: int64

In [38]:
b4_sign_df.loc[(b4_sign_df.latest_utm_source == 'None'),
               'latest_utm_source'] = b4_sign_df.loc[(b4_sign_df.latest_utm_source == 'None'),'utm_source']

In [39]:
b4_sign_df.latest_utm_source.value_counts(dropna = False)

baidu     25418
None      17986
others     2500
google      336
wechat      302
Name: latest_utm_source, dtype: int64

### for utm_media

In [40]:
b4_sign_df[(b4_sign_df.utm_medium.notnull())].event.value_counts(dropna = False)

$pageview           5422
profile_set_once    4202
Name: event, dtype: int64

In [41]:
b4_sign_df.utm_medium.value_counts(dropna = False)

NaN           36918
cpc            7239
mcpc           1616
mfeed           745
default          19
production        5
Name: utm_medium, dtype: int64

In [42]:
b4_sign_df[(b4_sign_df.latest_utm_medium.notnull())].event.value_counts(dropna = False)

$pageview                11509
btnClick                  5054
index_leave               4682
demo_leave                 608
about_leave                356
formSubmit                 352
courses_leave              284
click_send_cellphone       273
courses_play_leave         271
verify_cellphone_code      262
clickSubmit                212
page_close                  40
Name: event, dtype: int64

In [43]:
b4_sign_df.latest_utm_medium.value_counts(dropna = False)

NaN                   22639
cpc                   20657
mcpc                   2153
mfeed                   620
default                 283
answer                   72
hao.caibaojian.com       29
banner                   23
referral                 15
production               15
toutiao.io               13
youjian                   7
customer_m                5
lixinya                   5
sidonghui                 4
sidashidai                2
Name: latest_utm_medium, dtype: int64

In [44]:
b4_sign_df.loc[(b4_sign_df.latest_utm_medium.isnull()),
               'latest_utm_medium'] = b4_sign_df.loc[(b4_sign_df.latest_utm_medium.isnull()),'utm_medium']

### for latest_utm_medium
    google most frequently used medium

In [45]:
b4_sign_df.latest_utm_medium.fillna('None',inplace = True)

In [46]:
# b4_sign_df[(b4_sign_df.latest_utm_medium.notnull())].event.value_counts(dropna = False)

In [47]:
b4_sign_df.loc[(b4_sign_df.latest_utm_medium.apply(lambda x: x not in ['cpc','None','mcpc','mfeed','default'])),
               'latest_utm_medium'] = 'other'

In [48]:
b4_sign_df.latest_utm_medium.value_counts(dropna = False)

cpc        23789
None       18437
mcpc        2898
mfeed        941
default      286
other        191
Name: latest_utm_medium, dtype: int64

### for u'session_referrer_host' just keep it

In [49]:
# b4_sign_df.session_referrer_host.value_counts(dropna = False)

In [50]:
# b4_sign_df[(b4_sign_df.session_referrer_host.notnull())].event.value_counts()

    waiting for being imputed by others

### for u'referrer_host', keep it alone

In [51]:
b4_sign_df.referrer_host.value_counts(dropna = False)

NaN                                       30718
www.sensorsdata.cn                         7397
www.baidu.com                              4250
sensorsdata.cn                             1525
m.baidu.com                                 582
www.sogou.com                               240
36kr.com                                    166
localhost                                   151
www.google.com                              120
wap.sogou.com                               106
www.google.com.hk                           100
link.zhihu.com                               96
m.sogou.com                                  94
image.baidu.com                              53
www.so.com                                   38
googleads.g.doubleclick.net                  35
www.google.co.jp                             34
47.93.117.49                                 27
www.woshipm.com                              26
zhengbangchuangyi.cloud.sensorsdata.cn       20
cn.bing.com                             

In [52]:
b4_sign_df[(b4_sign_df.referrer_host.notnull())].event.value_counts()

$pageview    15824
Name: event, dtype: int64

In [53]:
b4_sign_df.referrer_host.fillna('None',inplace = True)

In [54]:
b4_sign_df.loc[(b4_sign_df.referrer_host.apply(lambda x: 'baidu' in x)),'referrer_host'] = 'baidu'

In [55]:
b4_sign_df.loc[(b4_sign_df.referrer_host.apply(lambda x: 'google' in x)),'referrer_host'] = 'google'

In [56]:
b4_sign_df.loc[(b4_sign_df.referrer_host.apply(lambda x: 'sogou' in x)),'referrer_host'] = 'sogou'

In [57]:
b4_sign_df.loc[(b4_sign_df.referrer_host.apply(lambda x: 'bing' in x)),'referrer_host'] = 'bing'

In [58]:
b4_sign_df.loc[(b4_sign_df.referrer_host.apply(lambda x: 'sensors' in x)),'referrer_host'] = 'sensors'

In [59]:
b4_sign_df.loc[(b4_sign_df.referrer_host.apply(lambda x: '36kr' in x)),'referrer_host'] = '36kr'

In [60]:
b4_sign_df.loc[(b4_sign_df.referrer_host.apply(lambda x: 'zhihu' in x)),'referrer_host'] = 'zhihu'

In [61]:
# b4_sign_df.loc[(b4_sign_df.referrer_host.apply(lambda x: 'github' in x)),'referrer_host'] = 'github'

In [62]:
b4_sign_df.loc[(
        b4_sign_df.referrer_host.apply(
            lambda x: 
            x not in ['google','None', 'baidu','sogou','36kr','localhost','sensors','zhihu'])),
               'referrer_host'] = 'others'

In [63]:
b4_sign_df.referrer_host.value_counts(dropna = False)

None         30718
sensors       9076
baidu         4970
others         555
sogou          455
google         322
36kr           199
localhost      151
zhihu           96
Name: referrer_host, dtype: int64

In [64]:
b4_sign_df.rename(columns = {'referrer_host':'ref_host_pageview'}, inplace = True)

### for u'referrHostUrl', to impute url_to_submit, and session_referrer_host, and drop it.

In [65]:
b4_sign_df.referrHostUrl.value_counts(dropna = False)

NaN                   46358
www.baidu.com           148
www.sogou.com            13
www.so.com                4
m.baidu.com               3
link.zhihu.com            2
www.google.com.hk         2
class.sanjieke.cn         1
blog.csdn.net             1
qiye.aliyun.com           1
cn.bing.com               1
blog.tingyun.com          1
shimo.im                  1
segmentfault.com          1
newseed.pedaily.cn        1
sc.vipkid.com.cn          1
youxuan.baidu.com         1
so.m.sm.cn                1
www.tisiwi.com            1
Name: referrHostUrl, dtype: int64

In [66]:
b4_sign_df[(b4_sign_df.referrHostUrl.notnull())].event.value_counts()

formSubmit    184
Name: event, dtype: int64

In [67]:
b4_sign_df.session_referrer_host.value_counts(dropna = False)

www.baidu.com                             20045
NaN                                       17814
www.sensorsdata.cn                         1291
36kr.com                                   1087
www.sogou.com                              1066
m.baidu.com                                1063
www.google.com                              648
www.google.com.hk                           542
sensorsdata.cn                              380
www.so.com                                  243
link.zhihu.com                              209
cn.bing.com                                 155
www.google.co.jp                            147
wap.sogou.com                               137
m.sogou.com                                 124
www.woshipm.com                             118
image.baidu.com                              74
class.sanjieke.cn                            56
www.mulumu.com                               54
passport.36kr.com                            53
googleads.g.doubleclick.net             

In [68]:
b4_sign_df.loc[(b4_sign_df.session_referrer_host.isnull()),
               'session_referrer_host'] = b4_sign_df.loc[(b4_sign_df.session_referrer_host.isnull()),'referrHostUrl']

In [69]:
b4_sign_df.session_referrer_host.value_counts(dropna = False)

www.baidu.com                             20045
NaN                                       17812
www.sensorsdata.cn                         1291
36kr.com                                   1087
www.sogou.com                              1066
m.baidu.com                                1063
www.google.com                              648
www.google.com.hk                           542
sensorsdata.cn                              380
www.so.com                                  243
link.zhihu.com                              209
cn.bing.com                                 155
www.google.co.jp                            147
wap.sogou.com                               137
m.sogou.com                                 124
www.woshipm.com                             118
image.baidu.com                              74
class.sanjieke.cn                            56
www.mulumu.com                               54
passport.36kr.com                            53
googleads.g.doubleclick.net             

In [70]:
b4_sign_df.url_to_submit.value_counts(dropna = False)

None           46035
baidu            407
demo              64
sogou             26
zhihu              4
wechat             2
tianguang          2
sanjieke.cn        2
Name: url_to_submit, dtype: int64

In [71]:
b4_sign_df.loc[(b4_sign_df.url_to_submit == 'None'),
               'url_to_submit'] = b4_sign_df.loc[(b4_sign_df.url_to_submit == 'None'),'referrHostUrl']

In [72]:
b4_sign_df.url_to_submit.value_counts(dropna = False)

NaN                   46006
baidu                   407
demo                     64
sogou                    26
www.baidu.com            17
zhihu                     4
www.so.com                3
www.google.com.hk         2
wechat                    2
tianguang                 2
sanjieke.cn               2
qiye.aliyun.com           1
link.zhihu.com            1
shimo.im                  1
blog.tingyun.com          1
newseed.pedaily.cn        1
cn.bing.com               1
www.tisiwi.com            1
Name: url_to_submit, dtype: int64

In [73]:
b4_sign_df.url_to_submit.fillna('None',inplace = True)

In [74]:
b4_sign_df.loc[(b4_sign_df.url_to_submit.apply(lambda x: 'baidu' in x)), 'url_to_submit'] = 'baidu'

In [75]:
b4_sign_df.loc[(b4_sign_df.url_to_submit.apply(lambda x: x not in ['None', 'demo', 'sogou', 'baidu'])), 'url_to_submit'] = 'others'

### for first_referrer_host', impute session_referr_host, drop it.

In [76]:
# b4_sign_df.first_referrer_host.value_counts(dropna = False)

In [77]:
b4_sign_df.session_referrer_host.value_counts(dropna = False)

www.baidu.com                             20045
NaN                                       17812
www.sensorsdata.cn                         1291
36kr.com                                   1087
www.sogou.com                              1066
m.baidu.com                                1063
www.google.com                              648
www.google.com.hk                           542
sensorsdata.cn                              380
www.so.com                                  243
link.zhihu.com                              209
cn.bing.com                                 155
www.google.co.jp                            147
wap.sogou.com                               137
m.sogou.com                                 124
www.woshipm.com                             118
image.baidu.com                              74
class.sanjieke.cn                            56
www.mulumu.com                               54
passport.36kr.com                            53
googleads.g.doubleclick.net             

In [78]:
b4_sign_df.loc[(b4_sign_df.session_referrer_host.isnull()),
               'session_referrer_host'] = b4_sign_df.loc[(b4_sign_df.session_referrer_host.isnull()),'first_referrer_host']

In [79]:
b4_sign_df.session_referrer_host.value_counts(dropna = False)

www.baidu.com                             23104
NaN                                       13089
m.baidu.com                                1529
www.sensorsdata.cn                         1339
www.sogou.com                              1278
36kr.com                                   1240
www.google.com                              697
www.google.com.hk                           600
sensorsdata.cn                              383
link.zhihu.com                              277
www.so.com                                  271
wap.sogou.com                               230
m.sogou.com                                 202
cn.bing.com                                 164
www.google.co.jp                            161
www.woshipm.com                             138
image.baidu.com                             110
googleads.g.doubleclick.net                  84
class.sanjieke.cn                            65
zhidao.baidu.com                             61
passport.36kr.com                       

In [80]:
b4_sign_df.session_referrer_host.fillna('None', inplace = True)

In [81]:
b4_sign_df.loc[(b4_sign_df.session_referrer_host.apply(lambda x: 'baidu' in x)), 'session_referrer_host'] = 'baidu'

In [82]:
b4_sign_df.loc[(b4_sign_df.session_referrer_host.apply(lambda x: 'sogou' in x)), 'session_referrer_host'] = 'sogou'

In [83]:
b4_sign_df.loc[(b4_sign_df.session_referrer_host.apply(lambda x: 'google' in x)), 'session_referrer_host'] = 'google'

In [84]:
b4_sign_df.loc[(b4_sign_df.session_referrer_host.apply(lambda x: 'zhihu' in x)), 'session_referrer_host'] = 'zhihu'

In [85]:
b4_sign_df.loc[(b4_sign_df.session_referrer_host.apply(lambda x: 'bing' in x)), 'session_referrer_host'] = 'bing'

In [86]:
b4_sign_df.loc[(b4_sign_df.session_referrer_host.apply(lambda x: 'sensors' in x)), 'session_referrer_host'] = 'sensors'

In [87]:
b4_sign_df.loc[(
        b4_sign_df.session_referrer_host.apply(
            lambda x: x not in ['None', 'baidu', 'sogou', 'google','zhihu','bing','36kr','bing','sensors'])), 
               'session_referrer_host'] = 'others'

### for u'company', u'contact', u'email', name, if fill out form to sign up

### for u'url_to_submit'(from_url) just keep it

In [88]:
b4_sign_df.url_to_submit.value_counts(dropna = False)

None      46006
baidu       424
demo         64
sogou        26
others       22
Name: url_to_submit, dtype: int64

### for u'isMsg' only in formSubmit ???

In [89]:
# b4_sign_df.isMsg.value_counts(dropna = False)

In [90]:
# b4_sign_df[(b4_sign_df.isMsg.notnull())].event.value_counts(dropna = False)

In [91]:
# b4_sign_df[(b4_sign_df.isMsg == True)].isSuccess.value_counts(dropna = False)

In [92]:
# b4_sign_df[(b4_sign_df.isMsg == False)].isSuccess.value_counts(dropna = False)

### for isSuccess just keep it, count # failure and wether it successed at last

### for name in btnClick just use it when name == request, set others aside

In [93]:
b4_sign_df[(b4_sign_df.name.notnull())].event.value_counts()

btnClick       8286
formSubmit      502
clickSubmit     291
Name: event, dtype: int64

In [94]:
b4_sign_df[(b4_sign_df.event == 'btnClick')].name.value_counts(dropna = False)

request     1891
document    1314
demo        1280
product     1211
b-round      814
about        588
blog         496
viedo        361
logo         278
mail          53
Name: name, dtype: int64

In [95]:
b4_sign_df[b4_sign_df.event == '$pageview'].current_page.value_counts(dropna = False)

homepage      8345
lookmanual    1965
demo          1867
manual        1701
product       1595
index         1421
bround        1158
about          770
courses        752
play_video     547
tool           151
share           20
Name: current_page, dtype: int64

### for u'btnClick_page'

In [97]:
b4_sign_df.btnClick_page.value_counts(dropna = False)

None             39574
index             5143
demo               853
about              526
courses_index      446
Name: btnClick_page, dtype: int64

In [99]:
b4_sign_df[(b4_sign_df.btnClick_page.notnull())].event.value_counts()

$pageview                20292
btnClick                  8286
index_leave               6693
profile_set_once          6481
demo_leave                1339
about_leave                653
courses_leave              594
formSubmit                 533
courses_play_leave         488
click_send_cellphone       395
verify_cellphone_code      363
clickSubmit                291
page_close                 134
Name: event, dtype: int64

### for set aside u'pagePosition',  add up pageStayTime 

In [100]:
b4_sign_df.pagePosition.value_counts(dropna = False)

NaN     36775
 1.0     5971
 2.0     2626
 4.0      590
 3.0      580
Name: pagePosition, dtype: int64

In [101]:
b4_sign_df[['pagePosition','pageStayTime','event']]

,pagePosition,pageStayTime,event
0,2.0,5.692,index_leave
1,NaN,NaN,profile_set_once
2,NaN,NaN,$pageview
3,NaN,NaN,btnClick
4,NaN,NaN,btnClick
5,1.0,21.291,demo_leave
6,NaN,NaN,btnClick
7,NaN,NaN,$pageview
8,NaN,NaN,btnClick
9,NaN,NaN,btnClick


### for u'project_name'

In [102]:
b4_sign_df.project_name.value_counts(dropna = False)

NaN           44871
default        1352
production      307
hunter           12
Name: project_name, dtype: int64

In [104]:
b4_sign_df[(b4_sign_df.project_name.notnull())].distinct_id.nunique()

88

In [103]:
b4_sign_df[(b4_sign_df.project_name.notnull())].event.value_counts()

$pageview                960
btnClick                 323
index_leave              245
demo_leave                85
page_close                18
about_leave               17
courses_play_leave        10
courses_leave              9
verify_cellphone_code      2
click_send_cellphone       2
Name: event, dtype: int64

In [105]:
b4_sign_df[(b4_sign_df.distinct_id == '0af0ee8e79705baa47156239e537b00750d9e919')]['project_name']

233    NaN
234    NaN
Name: project_name, dtype: object

### for u'result', wrong_num, less_than_1min, send_success.

In [106]:
b4_sign_df[(b4_sign_df.event == 'click_send_cellphone')].result.value_counts(dropna = False)

ajax success      349
手机号格式错误            43
距上次发送时间不到 1 分钟      3
Name: result, dtype: int64

In [107]:
b4_sign_df[(b4_sign_df.result == '手机号格式错误')].distinct_id.value_counts(dropna = False)

49914a389c53aa6c4d07fc0999f663dcacd900b5      2
81cb2c9dde1f55bd895f5e7ece5880ba2c2ce585      2
54d7b3ad4e7a56ac24370d9d8d1917eb70f6fa49      2
3348a66396a6a599a513d407e6e352186fbe57d4      1
95c887443e128504f165fca4e132de3d855a4cea      1
48d10ba62c0ee3ef6f66d84f67c8f62fc7ffcd4c      1
0fbad502076a89517b80f7b6773776d201ddfcc2      1
cacb3e7d7ce3ef8ce514319fec7011e2744bab99      1
f8d282d5c55ac9f661c28f8d59e028c1e37c2a23      1
b82dc696129aaa465d5a93095077d73f00258f80      1
bdd86ba8e24695531b651747c8e7fbb23965bf98      1
28caa9908b8720267dbf19db0ed3250fe12de950      1
7d24f773e21a9ad718c0d30c159435621e0ce71c      1
18c36368821ba19542b4304c50e4249e440ffd6f      1
19cb635bb4520c1315dda0a75fc721e3c7e2d249      1
0442857fa6eac7c50f5d72268a925e774ce2ed99      1
158fa1f5ab6367b3f7e2e937cfc619b0fbf07a91      1
40b41ede887727adb6ccf73a484d7454b5bb952c      1
ffbabac7e036dfa0896887d5a33ef1a970bf8394      1
870e185d90a112bf6e6d219eb359bd1b25ce2924      1
04b0e0ed4def01a7a2881d3d8aa55c341a36cbbc

In [108]:
b4_sign_df[(b4_sign_df.result == '距上次发送时间不到 1 分钟')].distinct_id.value_counts(dropna = False)

e3e17b041fcc15be7cb3cbae474ad39da4603441    2
e1cdb5a7ab9810606f8995906887a4ee3acd8950    1
Name: distinct_id, dtype: int64

In [109]:
b4_sign_df[(b4_sign_df.event == 'verify_cellphone_code')].result.value_counts(dropna = False)

正确           350
验证码错误         10
没有该号码验证信息      3
Name: result, dtype: int64

### wrong_code, no_code, verify_success

### for u'time', u'Bj_timestamp', u'dayofweek',u'hourofday', u'date'

### time zoom: morning, afternoon, evening (0/1)
#### weekday, Monday - Sunday, (0/1)

### for u'session_page_url', u'site_url', set them aside

In [110]:
b4_sign_df.session_page_url.value_counts(dropna = False)

homepage_utm               20837
homepage                   13177
None                        8378
demo                        1578
index                        930
course                       264
product                      227
local_host                   220
play_video                   202
bround                       173
homepage_ch=zhihu            162
about                        144
homepage_ch=demo             128
homepage_winzoom=1            32
homepage_from                 29
homepage_ch=chiyuan           25
share                         15
homepage_ch=36                12
homepage_ch=pedaily            7
homepage_ch=sangwenfeng        2
Name: session_page_url, dtype: int64

In [112]:
b4_sign_df.current_page.value_counts()

None          26250
homepage       8345
lookmanual     1965
demo           1867
manual         1701
product        1595
index          1421
bround         1158
about           770
courses         752
play_video      547
tool            151
share            20
Name: current_page, dtype: int64

In [111]:
b4_sign_df[(b4_sign_df.session_page_url != 'None')].event.value_counts(dropna = False)

$pageview                18474
btnClick                  8284
index_leave               6693
demo_leave                1339
about_leave                653
courses_leave              594
formSubmit                 533
courses_play_leave         488
click_send_cellphone       395
verify_cellphone_code      363
clickSubmit                291
page_close                  57
Name: event, dtype: int64

### u'url_on_page_close' combine with current_page

In [113]:
b4_sign_df.url_on_page_close.value_counts(dropna = False)

None                46408
tools_url_create       73
tools_code_auto        61
Name: url_on_page_close, dtype: int64

In [114]:
b4_sign_df[(b4_sign_df.url_on_page_close != 'None')].event.value_counts()

page_close    134
Name: event, dtype: int64

In [115]:
b4_sign_df.current_page.value_counts(dropna = False)

None          26250
homepage       8345
lookmanual     1965
demo           1867
manual         1701
product        1595
index          1421
bround         1158
about           770
courses         752
play_video      547
tool            151
share            20
Name: current_page, dtype: int64

In [116]:
b4_sign_df[(b4_sign_df.event == 'page_close')].current_page.value_counts(dropna = False)

None    134
Name: current_page, dtype: int64

In [117]:
b4_sign_df[(b4_sign_df.current_page == 'None')].url_on_page_close.value_counts(dropna = False)

None                26116
tools_url_create       73
tools_code_auto        61
Name: url_on_page_close, dtype: int64

In [118]:
b4_sign_df.loc[(b4_sign_df.current_page == 'None'),
               'current_page'] = b4_sign_df.loc[(b4_sign_df.current_page == 'None'),'url_on_page_close']

In [119]:
b4_sign_df.current_page.value_counts(dropna = False)

None                26116
homepage             8345
lookmanual           1965
demo                 1867
manual               1701
product              1595
index                1421
bround               1158
about                 770
courses               752
play_video            547
tool                  151
tools_url_create       73
tools_code_auto        61
share                  20
Name: current_page, dtype: int64

In [122]:
b4_sign_df.loc[(b4_sign_df.current_page.apply(lambda x: 'tools_' in x)),'current_page'] = 'tools_page_close'

In [123]:
b4_sign_df.current_page.value_counts(dropna = False)

None                26116
homepage             8345
lookmanual           1965
demo                 1867
manual               1701
product              1595
index                1421
bround               1158
about                 770
courses               752
play_video            547
tool                  151
tools_page_close      134
share                  20
Name: current_page, dtype: int64

## columns not useful:
    first_visit_time
    isMsg
    is_first_day, is_first_time
    u'browser_version, first_browser_language
    u'os_version', 
    u'latest_utm_term', latest_utm_campaign, latest_utm_content
    u'utm_campaign', u'utm_content',u'utm_term
    utm_source,utm_medium
    u'screen_height', u'screen_width'
    title,
    u'company', u'contact', u'email
    referrHostUrl, first_referrer_host
    model,
    url_on_page_close

In [124]:
b4_sign_df.to_csv('data/b4_sign_v2.csv', index = False, encoding = 'utf-8')

In [1]:
import pandas as pd

In [31]:
b4_sign_df = pd.read_csv('data/b4_sign_v2.csv')

In [32]:
b4_sign_df.name.fillna('None',inplace = True)

In [33]:
a = ['request','demo','product','document','b-round','about','blog','viedo','logo','mail','None']

In [34]:
b4_sign_df.loc[(b4_sign_df.name.apply(lambda x: x not in a)),'name'] = 'last_name'

In [5]:
b4_sign_df.name.value_counts(dropna = False)

None            37463
request          1891
cli_document     1314
cli_demo         1280
cli_product      1211
cli_bround        814
last_name         793
cli_about         588
cli_blog          496
cli_video         361
cli_logo          278
cli_mail           53
Name: name, dtype: int64

In [36]:
b4_sign_df.loc[(b4_sign_df.name == 'demo'),'name'] = 'cli_demo'

In [37]:
b4_sign_df.loc[(b4_sign_df.name == 'document'),'name'] = 'cli_document'

In [38]:
b4_sign_df.loc[(b4_sign_df.name == 'product'),'name'] = 'cli_product'

In [39]:
b4_sign_df.loc[(b4_sign_df.name == 'b-round'),'name'] = 'cli_bround'

In [40]:
b4_sign_df.loc[(b4_sign_df.name == 'about'),'name'] = 'cli_about'

In [41]:
b4_sign_df.loc[(b4_sign_df.name == 'blog'),'name'] = 'cli_blog'

In [42]:
b4_sign_df.loc[(b4_sign_df.name == 'viedo'),'name'] = 'cli_video'

In [43]:
b4_sign_df.loc[(b4_sign_df.name == 'logo'),'name'] = 'cli_logo'

In [44]:
b4_sign_df.loc[(b4_sign_df.name == 'mail'),'name'] = 'cli_mail'

In [46]:
b4_sign_df.drop(labels = ['first_visit_time','isMsg','is_first_day', 'is_first_time',
                          'browser_version', 'first_browser_language',
                          'os_version', 'latest_utm_term', 'latest_utm_campaign', 'latest_utm_content',
                          'utm_campaign', 'utm_content','utm_term',
                          'utm_source','utm_medium',
                          'title',u'company', u'contact', u'email',
                          'referrHostUrl', 'first_referrer_host',
                          'model','url_on_page_close'
                         ], 
                axis = 1, 
                inplace = True)

In [47]:
b4_sign_df.dayofweek = b4_sign_df.dayofweek + 1

In [48]:
b4_sign_df.to_csv('data/b4_sign_without_no_useful_col.csv',index = False, encoding = 'utf-8')

In [1]:
import pandas as pd
import numpy as np

In [2]:
b4_sign_df = pd.read_csv('data/b4_sign_without_no_useful_col.csv')

In [6]:
b4_sign_df.current_page.unique()

array(['None', 'homepage', 'manual', 'courses', 'index', 'play_video',
       'lookmanual', 'demo', 'product', 'about', 'bround', 'tool',
       'tools_page_close', 'share'], dtype=object)

In [7]:
b4_sign_df.btnClick_page.value_counts(dropna = False)

None             39574
index             5143
demo               853
about              526
courses_index      446
Name: btnClick_page, dtype: int64

In [8]:
b4_sign_df.loc[(b4_sign_df.btnClick_page == 'index'),'btnClick_page'] = 'btnpage_index'

In [9]:
b4_sign_df.loc[(b4_sign_df.btnClick_page == 'demo'),'btnClick_page'] = 'btnpage_demo'

In [10]:
b4_sign_df.loc[(b4_sign_df.btnClick_page == 'about'),'btnClick_page'] = 'btnpage_about'

In [11]:
b4_sign_df.loc[(b4_sign_df.btnClick_page == 'about'),'courses_index'] = 'btnpage_courses_index'

In [12]:
b4_sign_df.btnClick_page.value_counts(dropna = False)

None             39574
btnpage_index     5143
btnpage_demo       853
btnpage_about      526
courses_index      446
Name: btnClick_page, dtype: int64

In [15]:
b4_sign_df.name.unique()

array(['None', 'request', 'cli_document', 'cli_video', 'cli_blog',
       'last_name', 'cli_demo', 'cli_logo', 'cli_product', 'cli_about',
       'cli_bround', 'cli_mail'], dtype=object)

In [14]:
b4_sign_df.loc[(b4_sign_df.btnClick_page == 'None'), 'btnClick_page'] = b4_sign_df[(b4_sign_df.btnClick_page == 'None')].current_page

In [16]:
b4_sign_df.loc[(b4_sign_df.name == 'None'), 'name'] = b4_sign_df[(b4_sign_df.name == 'None')].event

In [17]:
pattern = b4_sign_df.groupby(by = 'distinct_id').apply(lambda x: x.sort_values('Bj_timestamp'))

In [20]:
pattern[['btnClick_page', 'name', 'event', 'current_page','Bj_timestamp']]

btnClick_page  \
distinct_id                                                        
00007ef910b6c9911f1b89d01a09aa3fc862f4a9 6317           homepage   
                                         6323      btnpage_index   
                                         6326               None   
                                         6334               None   
                                         6339               None   
                                         6340               None   
                                         6316               None   
000c46a27ef69fa22b56d253a9c72773338a1686 6277           homepage   
                                         6279      btnpage_index   
                                         6280               None   
                                         6380              index   
                                         6381      btnpage_index   
                                         6382      btnpage_index   
                                         6383      btnpage_index   
                                         7020      btnpage_index   
                                         7021      btnpage_index   
                                         7022      btnpage_index   
                                         7023               None   
                                         7026               None   
                                         7027               None   
                                         7028      btnpage_index   
                                         7032               None   
                                         7035            courses   
                                         7540      courses_index   
                                         7541               None   
                                         7543             bround   
                                         7556              index   
                                         7558             bround   
                                         7557               None   
                                         6276               None   
...                                                          ...   
ffc8ee900f28e2f14fd066413f1f74082125e6e1 11704              None   
ffc97d4a5d6396e902a05f53ae07ad98e7d3cd11 34929          homepage   
                                         34928              None   
ffd207edf81afd355fbba616480ccae95666a439 27513        lookmanual   
                                         27512              None   
ffdb2208bf05f87ffa460b8e61ff8c60d6b1582d 6576           homepage   
                                         6575               None   
ffe7a2a48843239c636c0ec54503e223531a03d2 29538              tool   
                                         29671  tools_page_close   
                                         29539              None   
ffe9025fc888f30546528e5e9f1cac24d1526476 45594          homepage   
                                         45595              None   
                                         45593              None   
fff9ee23119dac9d0379b2f8149cca8d31744d5c 32539          homepage   
                                         32546     btnpage_index   
                                         32547              None   
                                         32548             index   
                                         32549             about   
                                         32550     btnpage_about   
                                         32551              None   
                                         32552           product   
                                         32554              None   
                                         32555           product   
                                         32620              None   
                                         32538              None   
fffa276c4cc840ddb5993a52112b22542bd29c8b 18712          homepage   
                                       

In [21]:
b4_sign_df.to_csv('data/b4_sign_no_useless_col_eventAndname_btnclickpageAndcurrentpage.csv', index = False, encoding = 'utf-8')